In [3]:
import os
if "models" not in os.listdir("."):
    os.chdir("../..")


In [2]:
from penzai import pz
import json

from matplotlib import pyplot as plt
from tqdm.auto import tqdm, trange
import jax.numpy as jnp
import numpy as np
import random
from penzai.data_effects.side_output import SideOutputValue
from micrlhf.utils.activation_manipulation import add_vector


In [3]:
from micrlhf.llama import LlamaTransformer
llama = LlamaTransformer.from_pretrained("models/gemma-2-2b-it.gguf",
                                         from_type="gemma2",
                                         load_eager=True
                                         )

In [4]:
from transformers import AutoTokenizer
import jax
tokenizer = AutoTokenizer.from_pretrained("alpindale/gemma-2b")
tokenizer.padding_side = "right"

In [5]:
from sprint.task_vector_utils import load_tasks, ICLDataset, ICLSequence
tasks = load_tasks()

In [4]:
import json, numpy as np

with open("cleanup_results_gemma_2_all.jsonl", "r") as f:
    cleanup_results = [json.loads(line) for line in f]

In [5]:
task = "algo_last"
layer = 18

task_results = [result for result in cleanup_results if result["layer"] == layer]

print(len(task_results))

24


In [6]:
from micrlhf.utils.load_sae import sae_encode, get_dm_res_sae

sae = get_dm_res_sae(layer)
threshold = sae.get("threshold", 0)

print(threshold)

[3.788361  3.9346428 3.9553483 ... 3.785629  3.8830051 3.980076 ]


In [44]:
for r in task_results:
    task = r["task"]
    
    weights = np.array(task_results[0]["weights"])

    i = np.argwhere(weights > threshold).flatten()
    w = weights[i]

    idx = np.argsort(w)[::-1]

    i = i[idx]
    w = w[idx]

    print(
        f"TV loss: {task_results[0]['tv_loss']}"
    )
    print(
        f"FS loss: {task_results[0]['loss']}"
    )
    print(
        "ITO loss: ", task_results[0]["ito_loss"]
    )

[14456  1255  3383 11357 12551  6909]
[72.49621582 45.55445862 40.62357712 40.61974716 33.73713684 32.03196335]
TV loss: 3.515625
FS loss: 0.78125
ITO loss:  3.71875
